# Character-Level LSTM

Семинар основан на [материале](https://github.com/hse-ds/iad-deep-learning/blob/master/2021/seminars/sem08/sem08_task.ipynb) с майнора по DL.

На этом семинаре поговорим про рекуррентные нейронные сети (Recurrent Neural Networ, RNN). Мы обучим модель на тексте книги "Анна Каренина", после чего попробуем генерировать новый текст.

**Модель сможет генерировать новый текст на основе текста "Анны Карениной"!**

Можно посмотреть полезную [статью про RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) и [реализацию в Torch](https://github.com/karpathy/char-rnn).

Ообщая архитектура RNN:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="500">

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Загрузим данные

Загрузим текстовый файл "Анны Карениной".

In [3]:
# open text file and read in data as `text`
with open('anna.txt', 'r') as f:
    text = f.read()

Посмотрим первые 100 символов:

In [13]:
text[:200]

"Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverything was in confusion in the Oblonskys' house. The wife had\ndiscovered that the husband was carrying on"

### Токенизация

В ячейках ниже создадим два **словаря** для преобразования символов в целые числа и обратно. Кодирование символов как целых чисел упрощает их использование в качестве входных данных в сети.

In [6]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
print(chars)

# set(text) собирает уникальные символы (буквы, пробелы, знаки препинания и т. д.) из текста.
# tuple(set(text)) превращает множество в кортеж (упорядоченная структура).

('-', ')', 'J', 'i', '6', 'f', 'j', 'B', ',', '3', 'g', 'R', 'b', 'v', 'r', 'D', 'q', ' ', '/', 'H', 'Y', 'Z', 'x', 'V', '1', 'w', '(', 'd', 's', 'I', 'm', 'k', 'l', '*', 'X', 'p', 'y', 'W', 'z', 'a', 'P', 'M', 'O', 'U', '&', '8', '_', 'G', 'C', 'Q', '\n', 'S', 't', 'K', 'c', '7', '`', '4', 'h', '!', '2', 'E', '.', ':', ';', '5', 'A', "'", '"', 'N', 'e', '@', '9', '%', '$', 'T', 'n', 'F', '?', 'u', 'L', '0', 'o')


In [8]:
int2char = dict(enumerate(chars))
print(int2char)

# enumerate(chars) пронумеровывает каждый символ.
# Пример: {0: 'a', 1: 'b', 2: ' ', ...}.
# char2int: сопоставляет символ с числом.

{0: '-', 1: ')', 2: 'J', 3: 'i', 4: '6', 5: 'f', 6: 'j', 7: 'B', 8: ',', 9: '3', 10: 'g', 11: 'R', 12: 'b', 13: 'v', 14: 'r', 15: 'D', 16: 'q', 17: ' ', 18: '/', 19: 'H', 20: 'Y', 21: 'Z', 22: 'x', 23: 'V', 24: '1', 25: 'w', 26: '(', 27: 'd', 28: 's', 29: 'I', 30: 'm', 31: 'k', 32: 'l', 33: '*', 34: 'X', 35: 'p', 36: 'y', 37: 'W', 38: 'z', 39: 'a', 40: 'P', 41: 'M', 42: 'O', 43: 'U', 44: '&', 45: '8', 46: '_', 47: 'G', 48: 'C', 49: 'Q', 50: '\n', 51: 'S', 52: 't', 53: 'K', 54: 'c', 55: '7', 56: '`', 57: '4', 58: 'h', 59: '!', 60: '2', 61: 'E', 62: '.', 63: ':', 64: ';', 65: '5', 66: 'A', 67: "'", 68: '"', 69: 'N', 70: 'e', 71: '@', 72: '9', 73: '%', 74: '$', 75: 'T', 76: 'n', 77: 'F', 78: '?', 79: 'u', 80: 'L', 81: '0', 82: 'o'}


In [9]:
char2int = {ch: ii for ii, ch in int2char.items()}
print(char2int)

# Обратный словарь: { 'a': 0, 'b': 1, ' ': 2, ... }.

{'-': 0, ')': 1, 'J': 2, 'i': 3, '6': 4, 'f': 5, 'j': 6, 'B': 7, ',': 8, '3': 9, 'g': 10, 'R': 11, 'b': 12, 'v': 13, 'r': 14, 'D': 15, 'q': 16, ' ': 17, '/': 18, 'H': 19, 'Y': 20, 'Z': 21, 'x': 22, 'V': 23, '1': 24, 'w': 25, '(': 26, 'd': 27, 's': 28, 'I': 29, 'm': 30, 'k': 31, 'l': 32, '*': 33, 'X': 34, 'p': 35, 'y': 36, 'W': 37, 'z': 38, 'a': 39, 'P': 40, 'M': 41, 'O': 42, 'U': 43, '&': 44, '8': 45, '_': 46, 'G': 47, 'C': 48, 'Q': 49, '\n': 50, 'S': 51, 't': 52, 'K': 53, 'c': 54, '7': 55, '`': 56, '4': 57, 'h': 58, '!': 59, '2': 60, 'E': 61, '.': 62, ':': 63, ';': 64, '5': 65, 'A': 66, "'": 67, '"': 68, 'N': 69, 'e': 70, '@': 71, '9': 72, '%': 73, '$': 74, 'T': 75, 'n': 76, 'F': 77, '?': 78, 'u': 79, 'L': 80, '0': 81, 'o': 82}


In [12]:
# encode the text
encoded = np.array([char2int[ch] for ch in text])
encoded

# Каждый символ из текста заменяется на его числовое представление, используя словарь char2int.
# Результат сохраняется в массиве encoded (числовая версия текста).

array([48, 58, 39, ..., 28, 62, 50])

Посмотрим как символы закодировались целыми числами

In [11]:
encoded[:100]

array([48, 58, 39, 35, 52, 70, 14, 17, 24, 50, 50, 50, 19, 39, 35, 35, 36,
       17,  5, 39, 30,  3, 32,  3, 70, 28, 17, 39, 14, 70, 17, 39, 32, 32,
       17, 39, 32,  3, 31, 70, 64, 17, 70, 13, 70, 14, 36, 17, 79, 76, 58,
       39, 35, 35, 36, 17,  5, 39, 30,  3, 32, 36, 17,  3, 28, 17, 79, 76,
       58, 39, 35, 35, 36, 17,  3, 76, 17,  3, 52, 28, 17, 82, 25, 76, 50,
       25, 39, 36, 62, 50, 50, 61, 13, 70, 14, 36, 52, 58,  3, 76])

## Предобработка данных

Как можно видеть на изображении char-RNN выше, сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный словарь), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте создадим для этого функцию.

In [16]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [22]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 20)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


## Создаем mini-batch'и


Создадим мини-батчи для обучения. На простом примере они будут выглядеть так:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>
<br>

In [23]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''

    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total

    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))

    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Протестируем

Теперь создадим несколько наборов данных, и проверим, что происходит, когда мы создаем батчи.

In [24]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [27]:
# printing out the first 10 items in a sequence
print('x\n', x[:40, :40])
print('\ny\n', y[:40, :40])

x
 [[48 58 39 35 52 70 14 17 24 50 50 50 19 39 35 35 36 17  5 39 30  3 32  3
  70 28 17 39 14 70 17 39 32 32 17 39 32  3 31 70]
 [28 82 76 17 52 58 39 52 17 39 52 52 14 39 54 52 70 27 17 58 70 14 17 39
  52 52 70 76 52  3 82 76 17 25 39 28 17 58 70 14]
 [70 76 27 17 82 14 17 39 17  5 82 70  8 17 58 70 17 39 13 82  3 27 70 27
  17 58  3 28 17  5 39 52 58 70 14 62 17 19 70 50]
 [28 17 52 58 70 17 54 58  3 70  5 17 52 58 82 79 10 58 17 58  3 27 27 70
  76 50  3 76 52 70 14 70 28 52 17 82  5 17 58  3]
 [17 28 39 25 17 58 70 14 17 52 70 39 14  0 28 52 39  3 76 70 27  8 17 35
   3 52  3  5 79 32  8 17 28 25 70 70 52 17  5 39]
 [54 79 28 28  3 82 76 17 39 76 27 17 39 76 39 32 36 28  3 28  8 17 25 39
  28 17  3 76 17 35 14  3 76 54  3 35 32 70 17 27]
 [17 66 76 76 39 17 58 39 27 17 28 39  3 27 17 52 58 39 52 17 15 82 32 32
  36 17 25 82 79 32 27 17 70 22 54 79 28 70 17  3]
 [42 12 32 82 76 28 31 36 62 17 68  7 79 52 17 46 52 58 70 36 46 17 54 39
  76 76 82 52 17 10 14 39 28 35 17 52 58 39 52  

Если вы правильно реализовали get_batches, результат должен выглядеть примерно так:
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 хотя точные цифры могут отличаться. Убедитесь, что данные сдвинуты на один шаг для `y`!!!

---
## Зададим архитектуру


<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=500px>

### Структура модели

В `__init__` предлагаемая структура выглядит следующим образом:
* Создаваём и храним необходимые словари (уже релизовано)
* Определяем слой LSTM, который принимает в качестве параметров: размер ввода (количество символов), размер скрытого слоя `n_hidden`, количество слоев` n_layers`, вероятность drop-out'а `drop_prob` и логическое значение batch_first (True)
* Определяем слой drop-out с помощью drop_prob
* Определяем полносвязанный слой с параметрами: размер ввода `n_hidden` и размер выхода - количество символов
* Наконец, инициализируем веса

Обратите внимание, что некоторые параметры были названы и указаны в функции `__init__`, их нужно сохранить и использовать, выполняя что-то вроде` self.drop_prob = drop_prob`.

---
### Входы-выходы LSTM

Вы можете создать [LSTM layer](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) следующим образом

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
```

где `input_siz`e - это количество символов, которые эта ячейка ожидает видеть в качестве последовательного ввода, а `n_hidde`n - это количество элементов в скрытых слоях ячейки. Можно добавить drop-out, добавив параметр `dropout` с заданной вероятностью. Наконец, в функции `forward` мы можем складывать ячейки LSTM в слои, используя `.view`.

Также требуется создать начальное скрытое состояние всех нулей:

```python
self.init_hidden()
```

In [28]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else:
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [29]:
class CharRNN(nn.Module):

    def __init__(self,
                 tokens, # tokens: все уникальные символы (алфавит) в обучающем тексте.
                 n_hidden=256, # n_hidden: количество нейронов в скрытых слоях.
                                # число нейронов, используемых в каждом скрытом слое LSTM.
                                # Эти нейроны участвуют в обработке данных на каждом временном шаге.
                                # Как влияет? Большее число нейронов позволяет модели выучивать более сложные зависимости
                                # в данных, что особенно важно для работы с длинными последовательностями, такими как текст.
                                # Однако слишком большое значение может увеличить время обучения и риск переобучения.
                                # Пример: Если мы анализируем текст и пытаемся предсказать следующий символ,большое количество
                                # нейронов может помочь лучше учитывать контекст, например, длинные фразы или предложения.
                 
                 n_layers=2, # n_layers: количество слоев LSTM (Long Short-Term Memory, улучшенная версия RNN,
                             # которая лучше запоминает длинные зависимости в тексте).
                             # Количество слоев LSTM в модели. Каждый слой добавляет дополнительную "глубину",
                             # позволяя модели выучивать более сложные паттерны.
                             # Как влияет? Увеличение числа слоев может помочь сети обрабатывать данные
                             # на разных уровнях абстракции. Например:
                             # Нижний слой может анализировать простые связи (например, буквы и слоги).
                             # Верхние слои могут анализировать более сложные структуры
                             # (например, грамматические конструкции или общий смысл).
                             # Но слишком большое количество слоев может привести к "взрывающемуся"
                             # или "затухающему" градиенту, что усложняет обучение.
                             # Обычно: 2-3 слоя — это оптимальное число для большинства задач,
                             # поскольку оно балансирует сложность и вычислительную эффективность.
                 
                 drop_prob=0.5, # drop_prob: вероятность "выключения" некоторых нейронов (Dropout)
                                # для предотвращения переобучения.
                                # Dropout — это регуляризационная техника,
                                # при которой случайно "выключается" определённый процент нейронов во время обучения.
                                # Эта вероятность задаётся через параметр drop_prob.
                                # Как влияет? Это помогает предотвратить переобучение, заставляя модель искать
                                # более обобщённые решения, а не полагаться на конкретные нейроны.
                                # Например, если drop_prob = 0.5, то во время каждого шага обучения
                                # 50% нейронов будут отключены.
                                # Когда использовать? Dropout особенно полезен для глубоких моделей
                                # или небольших наборов данных, где переобучение является проблемой.
                 
                 lr=0.001 # lr: скорость обучения модели.
                          # Скорость обучения определяет, насколько сильно обновляются веса нейронной сети во время каждого шага обучения.
                          # Как влияет? Маленький lr: Сеть обучается медленно.
                          # Может найти более точное решение, но есть риск "застрять" в локальном минимуме.
                          # Большой lr: Сеть обучается быстро, но есть риск "проскочить" оптимальное решение.
                          # Обычно: Начинают с небольшого значения (например, 0.001) и корректируют его в процессе,
                          # используя адаптивные оптимизаторы (например, Adam, как в коде).

                ):
        
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # Словари преобразуют символы в числа (индексы) и наоборот, чтобы модель могла работать с текстом.
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the LSTM
        self.lstm = nn.LSTM(
            len(self.chars), # количество уникальных символов в тексте. Это задает размер входного слоя (input layer).
                             # У каждого символа в алфавите будет своё уникальное числовое представление.
                             # Например, если алфавит состоит из символов ['a', 'b', 'c', 'd'],
                             # то len(self.chars) = 4, и каждый символ будет закодирован как одно из чисел [0, 1, 2, 3].
            
            n_hidden, # количество нейронов (или размер "памяти") в каждом скрытом слое LSTM.
                      # Больше нейронов позволяет модели запоминать больше информации о последовательности,
                      # что полезно для сложных зависимостей в данных.
                      # Однако слишком большое значение увеличивает вычислительные затраты и риск переобучения.
                      # Оптимальный размер зависит от задачи: для простого текста может хватить 128-256 нейронов,
                      # а для сложных текстов — 512-1024.
            
            n_layers, # количество последовательных LSTM-слоев в модели.
                      # Более глубокие модели (с большим количеством слоев)
                      # позволяют лучше обрабатывать сложные зависимости в данных.
                      # Однако добавление слоев увеличивает риск затухания или взрыва градиентов
                      # (решается использованием Dropout или архитектур, как LSTM/GRU).
                      # Чаще всего используется 2-3 слоя, чтобы найти баланс между сложностью модели
                      # и вычислительной эффективностью.
            
            dropout=drop_prob,
            batch_first=True # Указывает, что входные данные должны иметь формат [batch_size, seq_length, features], где:
                             # batch_size — количество последовательностей в пакете.
                             # seq_length — длина каждой последовательности.
                             # features — размерность векторов на каждом временном шаге
                             # (например, 1, если это последовательность индексов символов).
        )

        ## Dropout случайным образом "выключает" нейроны (устанавливает их выход в 0) с вероятностью drop_prob.
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars)) # выходной слой сети,
                                                       # который преобразует выход скрытых слоев LSTM (размером n_hidden)
                                                       # в распределение вероятностей для каждого символа алфавита.

        ## Как это всё работает вместе?
        # 1. Входной текст преобразуется в последовательность индексов (длина — seq_length), соответствующих символам алфавита.
        # 2. Эти последовательности проходят через LSTM, где каждый временной шаг обновляет скрытое состояние модели.
        # Размер скрытого состояния задается параметром n_hidden, а глубина слоев — n_layers.
        # 3. На каждом шаге используется Dropout, чтобы случайно "выключать" нейроны и улучшать обобщающую способность модели.
        # 4. Полносвязный слой преобразует выход скрытых слоев в вероятности для каждого символа алфавита.
        # 5. На основе этих вероятностей выбирается следующий символ, который добавляется к последовательности.
        # Эта архитектура особенно популярна для задач, связанных с генерацией текста, таких как автозавершение,
        # написание стихов или генерация программного кода.

    # Forward-проход (прогон данных через модель)
    def forward(self, x, hidden):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## TODO: Get the outputs and the new hidden state from the lstm
        # Данные (x) проходят через LSTM, который возвращает:
            # r_output: выход LSTM для каждого временного шага.
            # hidden: новое скрытое состояние, которое нужно сохранить для следующего шага.
        r_output, hidden = self.lstm(x, hidden)

        ## Выход проходит через слой Dropout
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        # Выход преобразуется в вероятности для каждого символа с помощью полносвязного слоя:
        out = out.contiguous().view(-1, self.n_hidden) # contiguous и view изменяют форму данных для совместимости
                                                       # с полносвязным слоем.

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        # Модель возвращает:
            # out: вероятности символов.
            # hidden: обновленное состояние для следующего шага.
        return out, hidden

    ## Инициализация скрытых состояний
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        # Создаются тензоры для хранения скрытого состояния (hidden state) и состояния ячейки (cell state) LSTM.
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

## Обучим модель

Во время обучения нужно установить количество эпох, скорость обучения и другие параметры.

Используем оптимизатор Adam и кросс-энтропию, считаем loss и, как обычно, выполняем back propagation.

Пара подробностей об обучении:
> * Во время цикла мы отделяем скрытое состояние от его истории; на этот раз установив его равным новой переменной * tuple *, потому что скрытое состояние LSTM, является кортежем скрытых состояний.
* Мы используем [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) чтобы избавиться от взрывающихся градиентов.

### Концепция работы модели:
1. **Инициализация модели:**
    * Класс CharRNN является подклассом PyTorch nn.Module.
    * В конструкторе (__init__) определяются параметры модели и слои, такие как:
      * LSTM (основной слой, который обрабатывает последовательности символов).
      * Dropout (для регуляризации и предотвращения переобучения).
      * Полносвязный слой (для преобразования скрытых состояний в вероятности для каждого символа алфавита).
    * Также создаются словари (char2int и int2char) для преобразования символов в числовые индексы и обратно.

2. **Архитектура модели:**
    * LSTM слой обрабатывает последовательности входных данных, состоящих из числовых представлений символов.
    * После LSTM используется Dropout для регуляризации.
    * Полносвязный слой (self.fc) преобразует выходы LSTM в распределение вероятностей по символам алфавита.

3. **Прямой проход (forward):**
    * На вход подаются:
      * x — входные данные (батч последовательностей символов, закодированных в виде индексов).
      * hidden — начальные скрытые состояния LSTM.
    * Внутри метода:
      * Данные проходят через слой LSTM, который возвращает:
          * r_output — выходы LSTM для каждого временного шага.
          * Обновленное скрытое состояние hidden.
      * Затем применяется Dropout для улучшения обобщающей способности.
      * Полносвязный слой преобразует выходы LSTM в вероятности для каждого символа.
    * Метод возвращает:
      * out — вероятности символов (предсказания модели).
      * hidden — скрытое состояние для следующего шага.

4. **Инициализация скрытых состояний (init_hidden):**
      * Создает начальное скрытое состояние LSTM (нулевые тензоры) для первого прохода.

### Основные этапы выполнения кода:
1. **Создание экземпляра модели:**
    * Модель создается с заданными параметрами (например, количество слоев LSTM, скрытых нейронов, вероятность Dropout).
    * Уникальные символы текста (алфавит) передаются в качестве входных данных (tokens).

2. **Инициализация скрытых состояний:**
    * Перед началом обучения или генерации текста вызывается метод init_hidden, который создает нулевые тензоры для скрытых состояний.

3. **Прямой проход через модель (forward):**
    * Во время обучения или генерации входная последовательность символов преобразуется в индексы.
    * Эти индексы подаются в модель:
      * Сначала данные проходят через LSTM, который анализирует временные зависимости.
      * Результаты LSTM обрабатываются через Dropout для регуляризации.
      * Полносвязный слой преобразует выходы LSTM в вероятности каждого символа.

4. **Процесс обучения (описывается в функции train):**
    * Модель обучается на батчах данных. На каждом шаге:
      * Скрытое состояние инициализируется для текущего батча.
      * Батч данных проходит через модель.
      * Ошибка рассчитывается на основе разницы между предсказанными и реальными символами.
      * Градиенты вычисляются и обновляют веса модели.

5. **Генерация текста:**
    * После обучения модель может генерировать текст:
      * На основе входной строки (seed) предсказываются следующие символы.
      * Предсказанные символы добавляются в строку, и процесс повторяется.

### Порядок вызова функций:
1. **Создание модели:**
    * Конструктор __init__ инициализирует слои и параметры модели.

2. **Инициализация скрытых состояний:**
    * init_hidden вызывается перед началом обработки данных.

3. **Обработка данных:**
    * На каждом шаге:
      * Входные данные подаются в метод forward, где выполняется основной расчет.
      * Выходы используются для предсказаний или расчета ошибки.

4. **Обновление весов (во время обучения):**
    * После каждого шага обучения обновляются веса модели.

5. **Использование обученной модели:**
    * После обучения модель используется для генерации текста или предсказания символов.

### Логика поэтапного выполнения:
1. **Токенизация текста:**
    * Символы преобразуются в индексы для работы с LSTM.

2. **Обработка последовательностей:**
    * LSTM анализирует последовательность символов, учитывая контекст предыдущих временных шагов.

3. **Предсказание символов:**
    * Для каждого шага рассчитываются вероятности следующего символа.

4. **Генерация текста:**
    * С использованием обученной модели создаются новые последовательности текста, где каждый следующий символ выбирается на основе предсказанных вероятностей.

Модель подходит для задач генерации текста, таких как автозавершение или написание текстов в заданном стиле.

In [30]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    # 1. epochs — количество эпох. Эпоха — это полный проход по всему тренировочному набору данных (или его батчам).
    # Чем больше эпох, тем больше времени модель обучается на данных. Это позволяет ей лучше находить закономерности.
    # Однако слишком много эпох может привести к переобучению, когда модель начинает "запоминать" данные вместо того,
    # чтобы учиться обобщать. Нужно экспериментировать, чтобы найти оптимальное количество эпох.
    # Иногда удобно использовать Early Stopping, чтобы остановить обучение,
    # когда ошибка на валидационной выборке перестает снижаться.
    
    # 1. Почему одной эпохи недостаточно? За одну эпоху:
        # Модель видит каждый пример из датасета только один раз.
        # На этом этапе веса модели обновляются, но она еще не успевает выучить все особенности данных,
        # особенно если датасет сложный или большой.
        # Реальная модель учится постепенно, с каждым проходом улучшая свои предсказания.
        # Первая эпоха — это своего рода «разминка».

    # 2. Как работают несколько эпох? Каждая новая эпоха:
        # Использует обновленные веса модели с предыдущей эпохи.
        # Углубляет обучение: за счет постепенного уменьшения ошибки (loss), модель становится более точной.
        # Пример: В первую эпоху модель может выучить базовые паттерны, например:
        # "слово 'люблю' чаще встречается в позитивных текстах".
        # Во вторую эпоху она может начать учитывать более сложные зависимости,
        # например: "если 'люблю' стоит в отрицательном контексте ('не люблю'), то текст становится негативным".
        # В последующих эпохах модель уточняет свои предсказания и учится лучше обрабатывать сложные случаи.
    
    # 3. Что произойдет, если сделать всего 1 эпоху? Если остановиться после одной эпохи:
        # Модель может быть недообученной: она еще не успеет выучить все важные зависимости.
        # Ошибка (loss) на тестовых данных, скорее всего, останется высокой.
        # Это особенно критично на больших или сложных датасетах, где обучаться за одну эпоху просто невозможно.
    
    # 4. Сколько эпох нужно? Количество эпох зависит от:
        # Сложности задачи: Простые задачи требуют меньше эпох, сложные — больше.
        # Размеров датасета: Чем больше данных, тем больше времени нужно для обучения.
        # Переобучение: Если продолжать обучение слишком долго, модель начнет "зазубривать" данные,
        # что ухудшит её работу на новых данных.
        # Часто используется Early Stopping: обучение останавливается,
        # как только ошибка на валидационной выборке перестает уменьшаться.

    # 5. Пример с аналогией. Представь, что ты учишься готовить по новому рецепту:
        # Первая эпоха: Ты прочитал рецепт, попробовал приготовить блюдо. Получилось, но не идеально.
        # Вторая эпоха: Ты учел свои ошибки с первого раза (например, меньше соли, больше масла) и приготовил блюдо лучше.
        # Третья эпоха: Ты уже помнишь рецепт, понимаешь нюансы, и блюдо становится практически идеальным.
    
    # 2. batch_size — размер батча. Это количество примеров данных, которые обрабатываются моделью одновременно
    # перед обновлением весов. Маленькие батчи позволяют модели быстрее обновлять веса,
    # но могут приводить к шумным обновлениям (нестабильность градиента).
    # Большие батчи дают более стабильное обучение, но требуют больше памяти и времени для обработки.
    # Для небольших моделей можно использовать значения в диапазоне 32–128, а для более сложных задач — до 256–512,
    # если позволяет память.
    
    # 3. seq_length — длина последовательности символов. Это количество символов,
    # обрабатываемых моделью за один раз (последовательность временных шагов). Длинные последовательности позволяют модели
    # учитывать больше контекста, что важно для задач, где порядок символов критичен (например, генерация текста).
    # Слишком длинные последовательности могут замедлить обучение и увеличить требования к памяти.
    # Для текстов длиной 50-100 символов часто хватает seq_length = 100. Если текст имеет сложные долгосрочные зависимости,
    # длину можно увеличить.
    
    # 4. lr — скорость обучения
    # Скорость обучения (learning rate) определяет размер шага, с которым модель обновляет свои веса после вычисления градиента.
    # Слишком маленький lr приводит к медленному обучению, и модель может застрять в локальном минимуме.
    # Слишком большой lr может привести к пропуску минимума или нестабильности модели (разброс весов).
    # Обычно начинают с значения 0.001 или 0.01. Если модель не сходится, можно уменьшить lr.
    # Также популярны адаптивные методы, такие как Adam или SGD с динамическим изменением скорости обучения.
    
    # 5. clip — обрезка градиентов
    # Ограничение максимального значения градиентов до определенного порога.
    # В задачах с последовательностями (например, LSTM) градиенты могут стать очень большими (взрыв градиентов).
    # Это приводит к нестабильности и невозможности корректно обновлять веса.
    # Обрезка градиентов (gradient clipping) помогает стабилизировать процесс обучения, обрезая градиенты,
    # если они превышают заданное значение.
    # Часто используют значения 1.0 или 5.0. Эти значения нужно тестировать для конкретной задачи.
    
    # 6. val_frac — доля данных для проверки
    # Это часть данных, выделяемая для проверки (валидации), чтобы оценить, как хорошо модель обобщает данные.
    # Валидация позволяет понять, переобучается ли модель. Если ошибка на тренировочной выборке падает,
    # а на валидационной — растет, значит, модель начинает "запоминать" данные.
    # Обычно для валидации выделяют 10–20% от всех данных (то есть val_frac = 0.1 или 0.2).
    
    # Как все эти параметры работают вместе?
    # epochs и batch_size: Определяют, как долго и с каким количеством данных за раз модель будет обучаться. Например,
    # 10 эпох при batch_size=32 означают, что модель сделает 10 проходов по всему датасету, обрабатывая по 32 примера за раз.
    # seq_length: Влияет на то, сколько контекста модель учитывает за один временной шаг.
    # Длинные последовательности улучшают способность модели понимать связи между символами, но требуют больше ресурсов.
    # lr и clip: Управляют процессом обновления весов.
    # Правильная настройка скорости обучения и обрезки градиентов гарантируетстабильное обучение без разрывов.
    # val_frac: Позволяет отслеживать качество модели и избегать переобучения. Модель обучается на 80–90% данных,
    # а оставшиеся 10–20% используются для проверки.
    # Эти гиперпараметры взаимодействуют друг с другом, и их настройка зависит от задачи, размера данных и архитектуры модели.
    
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(train_on_gpu):
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())

                    val_losses.append(val_loss.item())

                net.train() # reset to train mode after iterationg through validation data

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Определим модель

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей.

In [31]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### Установим гиперпараметры

In [32]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2507... Val Loss: 3.1666
Epoch: 1/20... Step: 20... Loss: 3.1414... Val Loss: 3.1284
Epoch: 1/20... Step: 30... Loss: 3.1414... Val Loss: 3.1211
Epoch: 1/20... Step: 40... Loss: 3.1118... Val Loss: 3.1198
Epoch: 1/20... Step: 50... Loss: 3.1411... Val Loss: 3.1165
Epoch: 1/20... Step: 60... Loss: 3.1167... Val Loss: 3.1131
Epoch: 1/20... Step: 70... Loss: 3.1004... Val Loss: 3.1073
Epoch: 1/20... Step: 80... Loss: 3.1034... Val Loss: 3.0913
Epoch: 1/20... Step: 90... Loss: 3.0755... Val Loss: 3.0564
Epoch: 1/20... Step: 100... Loss: 2.9982... Val Loss: 2.9680
Epoch: 1/20... Step: 110... Loss: 2.9094... Val Loss: 2.8839
Epoch: 1/20... Step: 120... Loss: 2.8302... Val Loss: 2.7881
Epoch: 1/20... Step: 130... Loss: 2.7085... Val Loss: 2.6661
Epoch: 2/20... Step: 140... Loss: 2.6329... Val Loss: 2.5710
Epoch: 2/20... Step: 150... Loss: 2.5692... Val Loss: 2.5286
Epoch: 2/20... Step: 160... Loss: 2.5142... Val Loss: 2.4773
Epoch: 2/20... Step: 170... Loss:

Epoch: 10/20... Step: 1350... Loss: 1.3860... Val Loss: 1.4163
Epoch: 10/20... Step: 1360... Loss: 1.3937... Val Loss: 1.4184
Epoch: 10/20... Step: 1370... Loss: 1.3853... Val Loss: 1.4158
Epoch: 10/20... Step: 1380... Loss: 1.4261... Val Loss: 1.4127
Epoch: 10/20... Step: 1390... Loss: 1.4330... Val Loss: 1.4098
Epoch: 11/20... Step: 1400... Loss: 1.4342... Val Loss: 1.4086
Epoch: 11/20... Step: 1410... Loss: 1.4437... Val Loss: 1.4056
Epoch: 11/20... Step: 1420... Loss: 1.4258... Val Loss: 1.4032
Epoch: 11/20... Step: 1430... Loss: 1.3984... Val Loss: 1.4067
Epoch: 11/20... Step: 1440... Loss: 1.4186... Val Loss: 1.4026
Epoch: 11/20... Step: 1450... Loss: 1.3520... Val Loss: 1.4033
Epoch: 11/20... Step: 1460... Loss: 1.3676... Val Loss: 1.3994
Epoch: 11/20... Step: 1470... Loss: 1.3663... Val Loss: 1.3984
Epoch: 11/20... Step: 1480... Loss: 1.3834... Val Loss: 1.3946
Epoch: 11/20... Step: 1490... Loss: 1.3825... Val Loss: 1.3923
Epoch: 11/20... Step: 1500... Loss: 1.3674... Val Loss:

Epoch: 20/20... Step: 2660... Loss: 1.2317... Val Loss: 1.2784
Epoch: 20/20... Step: 2670... Loss: 1.2361... Val Loss: 1.2757
Epoch: 20/20... Step: 2680... Loss: 1.2185... Val Loss: 1.2754
Epoch: 20/20... Step: 2690... Loss: 1.2164... Val Loss: 1.2768
Epoch: 20/20... Step: 2700... Loss: 1.2255... Val Loss: 1.2754
Epoch: 20/20... Step: 2710... Loss: 1.1938... Val Loss: 1.2740
Epoch: 20/20... Step: 2720... Loss: 1.1941... Val Loss: 1.2752
Epoch: 20/20... Step: 2730... Loss: 1.1872... Val Loss: 1.2781
Epoch: 20/20... Step: 2740... Loss: 1.1846... Val Loss: 1.2759
Epoch: 20/20... Step: 2750... Loss: 1.1912... Val Loss: 1.2768
Epoch: 20/20... Step: 2760... Loss: 1.1919... Val Loss: 1.2796
Epoch: 20/20... Step: 2770... Loss: 1.2337... Val Loss: 1.2739
Epoch: 20/20... Step: 2780... Loss: 1.2620... Val Loss: 1.2765


## Checkpoint

После обучения сохраним модель, чтобы можно было загрузить ее позже. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [33]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Делаем предсказания

Теперь, когда мы обучили модель, сделаем предсказание следующих символов! Для предсказания мы передаем последний символ, и сеть предсказывает следующий символ, который мы потом передаем снова на вхол и получаем еще один предсказанный символ и так далее...

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые наиболее вероятные символы $K$. Это не позволит сети выдавать нам совершенно абсурдные прогнозы, а также позволит внести некоторый шум и случайность в выбранный текст. Узнать больше [можно здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [ ]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''

        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)

        if(train_on_gpu):
            inputs = inputs.cuda()

        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())

        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [ ]:
def sample(net, size, prime='The', top_k=None):

    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()

    net.eval() # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='Anna', top_k=5))

## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

### Полезные ссылки:


*   [Блог-пост Christopher'а Olah'а по LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)